In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
model = EfficientPhys()
model.compile(optimizer='adamw', loss='mse')
model(np.random.random((4, 160, 72, 72, 3)));
model.summary()

Model: "efficient_phys"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ tsm (TSM)                       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (640, 72, 72, 32)      │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (640, 70, 70, 32)      │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (640, 35, 35, 64)      │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (640, 33, 33, 64)      │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (640, 70, 70, 1)       │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_mask (Attention_mask) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (640, 33, 33, 1)       │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (640, 35, 35, 32)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (640, 35, 35, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (640, 128)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (640, 128)             │     2,097,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (640, 1)               │           129 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (640, 5184, 3)         │        20,736 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (640, 16384)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,183,811 (8.33 MB)

 Trainable params: 2,173,443 (8.29 MB)

 Non-trainable params: 10,368 (40.50 KB)

In [3]:
@jax.jit
@partial(shard_map, mesh=mesh, in_specs=P('x'), out_specs=P('x'))
def diff_label(x, y):
    y = ops.concatenate([y[:,1:]-y[:,:-1], ops.zeros_like(y[:,:1])], axis=1)
    return x, y

batch_size = 32

tape = "/root/ssd_cache/rppg_training_data/mmpd_160x72x72_all"

train_tape = load_datatape(tape, fold='train', batch=batch_size)
valid_tape = load_datatape(tape, fold='val', extended_hr='False', batch=batch_size)

#train_rlap = DatatapeMonitor(train_tape)
training_set, validation_set = KerasDataset(train_tape), KerasDataset(valid_tape)

training_set = training_set.apply_fn(compress_aug)
training_set = training_set.apply_fn(diff_label)
validation_set = validation_set.apply_fn(diff_label)

mmpd_160x72x72_all             datatape has been loaded.     7648 items total. fold=train selected
mmpd_160x72x72_all             datatape has been loaded.     1664 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/efficientphys_mmpd.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/efficientphys_mmpd.weights.h5')
model.load_weights('../weights/efficientphys_mmpd.weights.h5')

Training:   0%|          | 0/239 [00:00<?, ?it/s]

Validating:   0%|          | 0/52 [00:00<?, ?it/s]

Epoch   1:  Training loss:0.1197	Validation loss:0.1155	


Training:   0%|          | 0/239 [00:00<?, ?it/s]

Validating:   0%|          | 0/52 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/52 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.1162	Validation loss: 0.115	


Training:   0%|          | 0/239 [00:00<?, ?it/s]

Validating:   0%|          | 0/52 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.1162	Validation loss:0.1149	


In [5]:
eval_on_dataset(dataset_H5_rlap, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhys_MMPD_RLAP.h5', cumsum=True, scenes=['R1', 'R2', 'R3', 'R4'])
get_metrics('../results/EfficientPhys_MMPD_RLAP.h5')

  0%|          | 0/220 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,10.13±0.185,16.621±2.97,13.533±0.27,0.28803
Whole,5.386±0.368,7.665±2.925,7.263±0.549,0.65762


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhys_MMPD_COHFACE.h5', cumsum=True, fps=30)
get_metrics('../results/EfficientPhys_MMPD_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,12.488±0.33,18.836±4.183,18.647±0.533,0.19854
Whole,9.028±0.581,11.703±4.156,13.837±1.031,0.43101


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhys_MMPD_PURE.h5', cumsum=True)
get_metrics('../results/EfficientPhys_MMPD_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,13.647±0.669,22.969±6.392,20.745±1.047,0.43199
Whole,11.65±1.716,17.592±9.097,17.75±2.432,0.67138


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 160, (72, 72), step=4, batch=16, save='../results/EfficientPhys_MMPD_UBFC.h5', cumsum=True)
get_metrics('../results/EfficientPhys_MMPD_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,7.634±0.58,15.273±5.356,7.653±0.559,0.70582
Whole,6.788±1.253,10.586±6.12,6.825±1.194,0.84838
